# READ.ME

Here are two ways to evaluate my model.

No.1 is following all the phase.
No.2 is using road.tar.gz file.

But there is a complex step to make it clear,
so I suggest you just to see No.1,    
but use road.tar.gz file easily.

You can go section '1. Let's go with docker' directly.

# 0. Dataset

# Dataset downloading
Download the dataset for training and testing the object detector.

In [ ]:
#%cd djplace/practice/RoadDamageDetector/
!pwd

In [ ]:
# train
!wget https://mycityreport.s3-ap-northeast-1.amazonaws.com/02_RoadDamageDataset/public_data/IEEE_bigdata_RDD2020/train.tar.gz

!tar -xzvf train.tar.gz

# Dataset pre-processing
Remove the useless labels in the dataset.

In [ ]:
# In the train directory

%cd train

from sklearn.model_selection import train_test_split

import os
import xml.etree.ElementTree as ET
from shutil import copyfile
import argparse
from pathlib import Path
import random

!tree -d

In [ ]:
os.getcwd()
countries = ["Czech", "India", "Japan"]
labels = ["D00", "D10", "D20", "D40"]

for country in countries:
    annoFiles = os.listdir(os.path.join(os.getcwd(), country+"/annotations/xmls/"))
    jpgFiles = os.listdir(os.path.join(os.getcwd(), country+"/images/"))
    
    newCountry = "new_"+country
    Path(newCountry+"/Annotations").mkdir(parents=True, exist_ok=True)
    Path(newCountry+"/JPEGImages").mkdir(parents=True, exist_ok=True)
    
    for annoFile in annoFiles:
        tree = ET.parse(os.path.join(os.getcwd()+"/"+country+"/annotations/xmls/", annoFile))
        root = tree.getroot()
        for obj in root.findall("object"):
            a = obj.find("name").text
            if a not in labels:
                root.remove(obj)
            
        if len(root.findall("object")) > 0:
            tree.write(newCountry+"/Annotations/"+annoFile)
            copyfile(os.path.join(country+"/images/", annoFile.split(".")[0])+".jpg", newCountry+"/JPEGImages/"+annoFile.split(".")[0]+".jpg")
        else:
            continue

In [ ]:
# Copy them into the one directory for chaning to yolo format

Path("new_train/Annotations").mkdir(parents=True, exist_ok=True)
Path("new_train/JPEGImages").mkdir(parents=True, exist_ok=True)

!cp new_Czech/Annotations/*.xml new_train/Annotations/
!cp new_India/Annotations/*.xml new_train/Annotations/
!cp new_Japan/Annotations/*.xml new_train/Annotations/

!cp new_Czech/JPEGImages/*.jpg new_train/JPEGImages/
!cp new_India/JPEGImages/*.jpg new_train/JPEGImages/
!cp new_Japan/JPEGImages/*.jpg new_train/JPEGImages/

# Transform to YOLO format from VOC xml

In [ ]:
%cd ..

In [ ]:
annoFiles = os.listdir(os.path.join(os.getcwd(), "train/new_train/Annotations/"))
yoloFile = open("./xml2yolo_damage.txt", "w")
for i in range(len(annoFiles)):
    yoloFile.writelines(os.getcwd() + "/train/new_train/Annotations/" + annoFiles[i] + "\n")

yoloFile.close()

In [ ]:
!python xml2yolo.py --input-file xml2yolo_damage.txt

In [ ]:
Path("road/damage").mkdir(parents=True, exist_ok=True)

!cp train/new_train/JPEGImages/*.jpg road/damage/
!cp train/new_train/labels/*.txt road/damage/

# Split the dataset for training the dataset
Split the dataset to train dataset and validation dataset.

In [ ]:
!pwd

In [ ]:
# In the train directory

%cd train

!tree -d

In [ ]:
new_Czech_annoFiles = os.listdir(os.path.join(os.getcwd(), "new_Czech/JPEGImages/"))
print(len(new_Czech_annoFiles))

new_India_annoFiles = os.listdir(os.path.join(os.getcwd(), "new_India/JPEGImages/"))
print(len(new_India_annoFiles))

new_Japan_annoFiles = os.listdir(os.path.join(os.getcwd(), "new_Japan/JPEGImages/"))
print(len(new_Japan_annoFiles))

Czech_train, Czech_valid = train_test_split(new_Czech_annoFiles, test_size=0.2, random_state=42)
print(len(Czech_train), len(Czech_valid))

India_train, India_valid = train_test_split(new_India_annoFiles, test_size=0.2, random_state=42)
print(len(India_train), len(India_valid))

Japan_train, Japan_valid = train_test_split(new_Japan_annoFiles, test_size=0.2, random_state=42)
print(len(Japan_train), len(Japan_valid))

In [ ]:
# single source training dataset

file1 = open("../road/Cz_8_train.txt", "w")
for i in range(len(Czech_train)):
    file1.writelines('/usr/src/road/damage/' + Czech_train[i].split(".")[0] + '.jpg' + "\n")

file1.close()

file2 = open("../road/Cz_2_valid.txt", "w")
for i in range(len(Czech_valid)):
    file2.writelines('/usr/src/road/damage/' + Czech_valid[i].split(".")[0] + '.jpg' + "\n")

file2.close()

file1 = open("../road/In_8_train.txt", "w")
for i in range(len(India_train)):
    file1.writelines('/usr/src/road/damage/' + India_train[i].split(".")[0] + '.jpg' + "\n")

file1.close()

file2 = open("../road/In_2_valid.txt", "w")
for i in range(len(India_valid)):
    file2.writelines('/usr/src/road/damage/' + India_valid[i].split(".")[0] + '.jpg' + "\n")

file2.close()

file1 = open("../road/Ja_8_train.txt", "w")
for i in range(len(Japan_train)):
    file1.writelines('/usr/src/road/damage/' + Japan_train[i].split(".")[0] + '.jpg' + "\n")

file1.close()

file2 = open("../road/Ja_2_valid.txt", "w")
for i in range(len(Japan_valid)):
    file2.writelines('/usr/src/road/damage/' + Japan_valid[i].split(".")[0] + '.jpg' + "\n")

file2.close()

In [ ]:
# mix source training dataset

CzInJa_train = new_Czech_annoFiles + new_India_annoFiles + Japan_train
random.shuffle(CzInJa_train)

CzInJa_valid = Japan_valid
random.shuffle(CzInJa_valid)

file1 = open("../road/CzInJa_train.txt", "w")
for i in range(len(CzInJa_train)):
    file1.writelines('/usr/src/road/damage/' + CzInJa_train[i].split(".")[0] + '.jpg' + "\n")

file1.close()

file2 = open("../road/CzInJa_valid.txt", "w")
for i in range(len(CzInJa_valid)):
    file2.writelines('/usr/src/road/damage/' + CzInJa_valid[i].split(".")[0] + '.jpg' + "\n")

file2.close()

japanHalfLength = len(new_Japan_annoFiles)/2

JaCz_train = new_Japan_annoFiles[:int(japanHalfLength)] + Czech_train
random.shuffle(JaCz_train)

file1 = open("../road/Ja5Cz8_train.txt", "w")
for i in range(len(JaCz_train)):
    file1.writelines('/usr/src/road/damage/' + JaCz_train[i].split(".")[0] + '.jpg' + "\n")

file1.close()

file2 = open("../road/Ja5Cz8_valid.txt", "w")
for i in range(len(Czech_valid)):
    file2.writelines('/usr/src/road/damage/' + Czech_valid[i].split(".")[0] + '.jpg' + "\n")

file2.close()

JaIn_train = new_Japan_annoFiles[:int(japanHalfLength)] + India_train
random.shuffle(JaIn_train)

file1 = open("../road/Ja5In8_train.txt", "w")
for i in range(len(JaIn_train)):
    file1.writelines('/usr/src/road/damage/' + JaIn_train[i].split(".")[0] + '.jpg' + "\n")

file1.close()

file2 = open("../road/Ja5In8_valid.txt", "w")
for i in range(len(India_valid)):
    file2.writelines('/usr/src/road/damage/' + India_valid[i].split(".")[0] + '.jpg' + "\n")

file2.close()

In [ ]:
# for 5-Fold split, there will be 5 dataset pair, train & valid
# In the road directory,
# you can use this files to make more different train / valid dataset.
'''
!python split_Cz.py
!python split_In.py
!python split_Ja.py
'''

# 1. Let's go with docker
If you are not a docker user,    
just follow to set tu the YOLOv5 from their github.    
<https://github.com/ultralytics/yolov5>

In [ ]:
!pwd

In [ ]:
%cd ..

In [ ]:
!tar -xzvf road.tar.gz

In [ ]:
!sudo docker run --ipc=host --gpus all -it -v "$(pwd)"/road:/usr/src/road ultralytics/yolov5:latest

In [ ]:
!mv /usr/src/road/road-*.yaml /usr/src/app/data/
!mv /usr/src/road/yolov5x-road.yaml /usr/src/app/models/
!mv /usr/src/road/road_detect.py /usr/src/app/
!mv /usr/src/road/merge.py /usr/src/app/

# 2. Train
I did various experiments, and I could get some good models.    
This is just an example, there were many mixed dataset,    
and failed weight files, too.

If you can't use this ipynb file in docker,    
use just command without ! or % in the terminal.

In [ ]:
%cd /usr/src/app

In [ ]:
# Train YOLOv5x on Czech, India and Japan road damage dataset
!python train.py --img 704 --batch 8 --epochs 50 --data ../road/road-Cz.yaml --cfg ../road/yolov5x-road.yaml --weights yolov5x.pt
!python train.py --img 704 --batch 8 --epochs 50 --data ../road/road-In.yaml --cfg ../road/yolov5x-road.yaml --weights yolov5x.pt
!python train.py --img 608 --batch 16 --epochs 50 --data ../road/road-Ja.yaml --cfg ../road/yolov5x-road.yaml --weights yolov5x.pt

# for improving the GPU utilization, two V100
#!python -m torch.distributed.launch --nproc_per_node 2 train.py --img 704 --batch 8 --epochs 50 --data ../road/road-Cz.yaml --cfg ../road/yolov5x-road.yaml --weights yolov5x.pt
#!python -m torch.distributed.launch --nproc_per_node 2 train.py --img 704 --batch 8 --epochs 50 --data ../road/road-In.yaml --cfg ../road/yolov5x-road.yaml --weights yolov5x.pt
#!python -m torch.distributed.launch --nproc_per_node 2 train.py --img 608 --batch 16 --epochs 50 --data ../road/road-Ja.yaml --cfg ../road/yolov5x-road.yaml --weights yolov5x.pt

In [ ]:
# weight files for inference
#!ls /usr/src/road/road_weights

# 3. Inference

In [ ]:
%cd /usr/src/app/inference

# test1
!wget https://mycityreport.s3-ap-northeast-1.amazonaws.com/02_RoadDamageDataset/public_data/IEEE_bigdata_RDD2020/test1.tar.gz
# test2
!wget https://mycityreport.s3-ap-northeast-1.amazonaws.com/02_RoadDamageDataset/public_data/IEEE_bigdata_RDD2020/test2.tar.gz

!tar -xzvf test1.tar.gz
!tar -xzvf test2.tar.gz

In [ ]:
%cd /usr/src/app

In [ ]:
# predict to the labels for each country test1 dataset

!python road_detect.py --source ./inference/test1/Czech/images/ --weights ../road/road_weights/Cz-1_best.pt ../road/road_weights/Cz-2_best.pt ../road/road_weights/Cz-3_best.pt ../road/road_weights/Cz-4_best.pt ../road/road_weights/Cz-5_best.pt ../road/road_weights/JaCz_best.pt --img-size 704 --conf-thres 0.4 --augment --save-txt
!python road_detect.py --source ./inference/test1/India/images/ --weights ../road/road_weights/In-5_best.pt ../road/road_weights/JaIn_best.pt --img-size 704 --conf-thres 0.4 --augment --save-txt
!python road_detect.py --source ./inference/test1/Japan/images/ --weights ../road/road_weights/Ja-1_best.pt ../road/road_weights/Ja-2_best.pt ../road/road_weights/Ja-3_best.pt ../road/road_weights/Ja-4_best.pt ../road/road_weights/Ja-5_best.pt ../road/road_weights/CzInJa_2_best.pt --img-size 608 --conf-thres 0.4 --augment --save-txt

In [ ]:
# merge them into one sampleSubmission.txt for test1
!python merge.py

In [ ]:
# predict to the labels for each country test2 dataset

!python road_detect.py --source ./inference/test2/Czech/images/ --weights ../road/road_weights/Cz-4_best.pt ../road/road_weights/JaCz_best.pt --img-size 704 --conf-thres 0.4 --augment --save-txt
!python road_detect.py --source ./inference/test2/India/images/ --weights ../road/road_weights/In-5_best.pt ../road/road_weights/JaIn_best.pt --img-size 704 --conf-thres 0.4 --augment --save-txt
!python road_detect.py --source ./inference/test2/Japan/images/ --weights ../road/road_weights/Ja-1_best.pt ../road/road_weights/Ja-2_best.pt ../road/road_weights/Ja-5_best.pt --img-size 608 --conf-thres 0.4 --augment --save-txt

In [ ]:
# merge them into one sampleSubmission.txt for test2
!python merge.py